In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os, sys
import itertools
import pickle
import h5py

# local imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils import *
from gtv import *
from preprocessing import *

# SST Observations

#### Download raw data [here](https://drive.google.com/drive/u/1/folders/1jQ3m8DI0m8Avl5dB3I2AacHuW1ZLWwpw)

The file 'SST_Pacific.mat' includes monthly observations of sea surface temperature (in Celsius) over the Pacific Ocean during 01/1940-12/2018, from the COBE SST v2 dataset.

Temperature over land is replaced with "NaN" values.

The Pacific has been defined as:
   - Latitudes: 60S-60N
   - Longitudes: 80E-280E

Particularly, the file 'SST_Pacific.mat' contains: 

   - 3D matrix 'SST_Pacif': 121 by 201 by 948
   - vector 'lat': 121 by 1 which corresponds to the latitudinal points over Pacific 
   - vector 'lon': 201 by 1 which corresponds to the longitudinal points over Pacific 
   - 2D matrix 'dates': 948 by 2 which corresponds to the dates  
   
   

In [21]:
# load .mat file
monthly_atmo = {}
with h5py.File('../data/SST_pacific.mat') as f:
    for k,v in f.items():
        monthly_atmo[k] = np.array(v)

In [22]:
# flatten 3D array to 2D dataframe
sst = monthly_atmo['SST_Pacif']
obs_lats = monthly_atmo['lat'][0]
obs_lons = monthly_atmo['lon'][0]
flat_df = flatten_series(sst, obs_lats, obs_lons, 'monthly', 79, 1940)
flat_df['var'] = 'sst'
flat_df['val'] = flat_df.temp

In [23]:
# drop nulls (over land)
df = flat_df
df = df.dropna()

In [24]:
# remove seasonal trend
sst = df[df['var']=='sst']
seasonal = sst.groupby(['lat', 'lon', 'month']).val.agg(['mean', 'std']).reset_index()
sst = pd.merge(sst, seasonal, on=['lat', 'lon', 'month'])
sst['anomaly'] = (sst['val'] - sst['mean'])/sst['std']
sst['val'] = sst['anomaly']

In [25]:
# aggregate to 10x10 temporal scale, stack into matrix, scale 
X_obs, fts_obs, mdf_obs = monthly_X(sst, step=10, agg=True, scale=False, max_year=2018)
X_obs = preprocessing.scale(signal.detrend(X_obs, axis=0))

# LENS

#### Download raw data [here](https://drive.google.com/drive/u/1/folders/1ddwOm4wIt6A8-_y9KxI0OdSTqTamhK8i)

The file 'surf_temp.mat' includes monthly data of surface temperature (in Kelvin) over the Pacific Ocean during 01/1920-12/2005, for all 40 ensembles from LENS.

The Pacific has been defined as:

   - Latitudes: 60S-60N
   - Longitudes: 80E-280E
   
Particularly, the file 'surf_temp.mat' contains: 

   - 4D matrix 'TS': 128 by 161 by 1032 by 40
   - vector 'lat': 128 by 1 which corresponds to the latitudinal points over Pacific 
   - vector 'lon': 161 by 1 which corresponds to the longitudinal points over Pacific 

In [4]:
# load .mat file
surf_temp = {}
with h5py.File('/Users/abbystevens/Downloads/surf_temp.mat', 'r') as f:
    for k, v in f.items():
        surf_temp[k] = np.array(v)

In [5]:
surf_temp['TS'].shape

(40, 1032, 161, 128)

First, we interpolate LENS onto the same grid as the observations

In [7]:
from scipy.interpolate import RegularGridInterpolator

lats = surf_temp['lat'][0].copy() #128
lons = surf_temp['lon'][0].copy() #161

# define grid to interpolate 
x = lons.copy()
y = lats.copy()
z = np.arange(surf_temp['TS'].shape[1])

# anchor bounds for imputation (quirk of imputation library)
x[0] = 79.5
y[-1] = 60.5

# great grid to interpolate onto
pts = np.array([i for i in itertools.product(z, obs_lons, obs_lats)])

iLENS = np.zeros((40, z.shape[0], obs_lons.shape[0], obs_lats.shape[0])) #initialize empty array

In [12]:
iLENS.shape

(40, 1032, 201, 121)

In [13]:
for i in range(40):
    if i%10==0: print(i)
    lens = surf_temp['TS'][i] # extract ith trajectory
    rgi = RegularGridInterpolator((z, x, y), lens) #train interpolator
    ilens = rgi(pts).reshape(z.shape[0], obs_lons.shape[0], obs_lats.shape[0]) #interpolate onto new points
    iLENS[i] = ilens

0
10
20
30


In [19]:
lens = iLENS[0]
flat_df = flatten_lens(lens, obs_lats, obs_lons, 1920)
Xlens, fts, lens_df = monthly_X(flat_df, step=10, agg=True, scale=False, min_year=1940, max_year=2005)
lens_df['trajectory'] = 0
for i in range(1, 40):
    if i%10==0: print(i)
    lens = iLENS[i]
    flat_df = flatten_lens(lens, obs_lats, obs_lons, 1920)
    X, fts, mdf = monthly_X(flat_df, step=10, agg=True, scale=False, min_year=1940, max_year=2005)
    mdf['trajectory'] = i
    lens_df = lens_df.append(mdf, ignore_index=True)
    Xlens = np.vstack([Xlens, X])
# save processed files
lens_df.to_csv('../data/lens_df_raw.csv', index=False)
pd.DataFrame(Xlens).to_csv('lens_X_raw.csv', index=False)
fts_lens = fts

10
20
30


#### Overlap LENS and Obs

In [48]:
fts_lens['ix_lens'] = fts_lens.index
fts_obs['ix_obs'] = fts_obs.index
fts = pd.merge(fts_lens, fts_obs)

# remove some more land
land = pd.DataFrame([[-25, 145],[ 35, 115],[45, 265]], columns = ['lat', 'lon'])
land['land'] = 1
fts = pd.merge(fts, land, how='left')
fts = fts[fts.land.isnull()].drop(columns='land')

fts = pd.merge(fts_old, fts_lens)

#### More LENS processing

In [716]:
lens_df = pd.read_csv('../data/lens_df_raw.csv')
Xlens = pd.read_csv('lens_X_raw.csv').values

In [32]:
# remove seasonal trend
seasonal = lens_df.groupby(['lat', 'lon', 'month', 'trajectory']).val.agg(['mean', 'std']).reset_index()
lens_df = pd.merge(lens_df, seasonal, on=['lat', 'lon', 'month', 'trajectory'])
lens_df['anomaly'] = (lens_df['val'] - lens_df['mean'])/lens_df['std']

In [53]:
# don't remove mean trend
t = lens_df[lens_df.trajectory==0]
Xlens = np.vstack(t.groupby('year').anomaly.apply(np.array))
for i in range(1, 40):
    t = lens_df[lens_df.trajectory==i]
    Xtmp = np.vstack(t.groupby('year').anomaly.apply(np.array))
    Xlens = np.vstack([Xlens, Xtmp])
    
# do remove mean trend
t = lens_df[lens_df.trajectory==0]
Xlens_dt = signal.detrend(np.vstack(t.groupby('year').anomaly.apply(np.array)), axis=0)
for i in range(1, 40):
    t = lens_df[lens_df.trajectory==i]
    Xtmp = signal.detrend(np.vstack(t.groupby('year').anomaly.apply(np.array)), axis=0)
    Xlens_dt = np.vstack([Xlens_dt, Xtmp])

In [54]:
pd.DataFrame(Xlens[:, fts.ix_lens]).to_csv('../data/Xlens_new_interpolated.csv', index=False)
pd.DataFrame(Xlens_dt[:, fts.ix_lens]).to_csv('../data/Xlens_new_interpolated_dt.csv', index=False)